In [2]:
import os, json, re, string, math
from collections import defaultdict
import numpy as np
from pathlib import Path
from typing import List, Dict

from sklearn.metrics.pairwise import cosine_similarity
import joblib
import nltk

In [3]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive


Mounted at /content/drive
'Colab Notebooks'	    'Google AI Studio'	 writings
'Document sans titre.gdoc'   Projet_SRI


In [4]:
# CONFIGURATION
BASE_FOLDER = "/content/drive/MyDrive/Projet_SRI"
INDEX_JSON_PATH = os.path.join(BASE_FOLDER, "index_inverse.json")
DOCS_FOLDER = os.path.join(BASE_FOLDER, "med_md")  # dossier des fichiers recettes
SAVE_FOLDER = os.path.join(BASE_FOLDER, "ri_model")
os.makedirs(SAVE_FOLDER, exist_ok=True)

In [5]:

# UTILS : prétraitement simple (pour la requete utilisateur (utiliser dans build_query_vector))

import nltk
try:
    nltk.data.find("tokenizers/punkt")
except:
    nltk.download("punkt")
try:
    nltk.data.find("corpora/stopwords")
except:
    nltk.download("stopwords")

from nltk.corpus import stopwords
french_stopwords = set(stopwords.words("french"))
!ls "/content/drive/MyDrive/Projet_SRI"

def clean_text(text: str) -> str:
    # basic cleaning: lowercase, remove punctuation (keep accents), collapse whitespace
    text = text.lower()
    # remove punctuation (but keep accented letters)
    text = re.sub(r"[{}]".format(re.escape(string.punctuation)), " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


 Indexation.ipynb     meta_data.json		  retrieved_list.py
 index_inverse.json  'ModeleVectoriel(1).ipynb'   ri_model
 medicaments_pdf      ModeleVectoriel.ipynb
 med_md		      qrels.py


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
# Charger les documents (.txt)  (uniquement pour pouvoir afficher les resultat(snippet...))

def load_documents(folder: str) -> Dict[str, str]:
    docs = {}
    for f in sorted(os.listdir(folder)):
        if f.endswith(".md"):
            with open(os.path.join(folder, f), "r", encoding="utf-8") as file:
                docs[f] = file.read()
    return docs

documents = load_documents(DOCS_FOLDER)
print(f"📂 Documents chargés : {len(documents)}")

📂 Documents chargés : 50


In [7]:
# Charger l’index inversé JSON

with open(INDEX_JSON_PATH, "r", encoding="utf-8") as f:
    inverted_index = json.load(f)

print(f"📘 Index inversé chargé : {len(inverted_index)} termes")


📘 Index inversé chargé : 3594 termes


In [8]:
# Construire les structures TF et IDF

doc_term_freq = defaultdict(dict)

for term, postings in inverted_index.items():
    for entry in postings:
        doc_id = entry["doc_id"]
        tf = entry["tf"]
        doc_term_freq[doc_id][term] = tf

N = len(doc_term_freq)
idf = {}
for term, postings in inverted_index.items():
    df = len(postings)
    idf[term] = math.log((N / df), 10) if df else 0.0

In [9]:
#Construire les vecteurs TF-IDF pour chaque document

tfidf_vectors = {}

for doc_id, term_freqs in doc_term_freq.items():
    tfidf_vectors[doc_id] = {}
    for term, tf in term_freqs.items():
        tfidf_vectors[doc_id][term] = tf * idf[term]

print("✅ Vecteurs TF-IDF générés pour tous les documents.")

✅ Vecteurs TF-IDF générés pour tous les documents.


In [10]:
#Sauvegarde du modèle dans ri_model ===
import joblib

tfidf_path = os.path.join(SAVE_FOLDER, "tfidf_vectors.pkl")
idf_path = os.path.join(SAVE_FOLDER, "idf.pkl")
index_path = os.path.join(SAVE_FOLDER, "inverted_index.pkl")
vocab_path = os.path.join(SAVE_FOLDER, "vocab.pkl")

joblib.dump(tfidf_vectors, tfidf_path)
joblib.dump(idf, idf_path)
joblib.dump(inverted_index, index_path)
joblib.dump(list(inverted_index.keys()), vocab_path)

print("💾 Modèle sauvegardé dans :", SAVE_FOLDER)


💾 Modèle sauvegardé dans : /content/drive/MyDrive/Projet_SRI/ri_model


In [11]:
# Calcul similarité cosinus

def cosine_similarity_dict(v1, v2):
    common = set(v1.keys()) & set(v2.keys())
    num = sum(v1[t] * v2[t] for t in common)
    denom1 = math.sqrt(sum(v**2 for v in v1.values()))
    denom2 = math.sqrt(sum(v**2 for v in v2.values()))
    return num / (denom1 * denom2) if denom1 and denom2 else 0

In [12]:
# Traitement de la requête utilisateur

def build_query_vector(query: str) -> Dict[str, float]:
    query = clean_text(query)
    tokens = [t for t in query.split() if t not in french_stopwords]
    tf_query = defaultdict(int)
    for t in tokens:
        tf_query[t] += 1
    q_vec = {}
    for term, tf in tf_query.items():
        if term in idf:
            q_vec[term] = tf * idf[term]
    return q_vec

In [13]:

# Fonction de recherche

def get_snippet(doc_text: str, query: str, window: int = 200) -> str:
    q_tokens = [t for t in query.lower().split() if t not in french_stopwords]
    lowered = doc_text.lower()
    for token in q_tokens:
        pos = lowered.find(token)
        if pos != -1:
            start = max(0, pos - window//2)
            end = min(len(doc_text), pos + window//2)
            snippet = doc_text[start:end]
            snippet = re.sub(r"\s+", " ", snippet)
            return "..." + snippet + "..."
    return re.sub(r"\s+", " ", doc_text[:window]) + "..."

def search(query: str, top_k: int = 5):
    q_vec = build_query_vector(query)
    if not q_vec:
        print("⚠️ Aucun des termes de la requête n'existe dans l'index.")
        return []

    scores = {}
    for doc_id, d_vec in tfidf_vectors.items():
        score = cosine_similarity_dict(q_vec, d_vec)
        if score > 0:
            scores[doc_id] = score

    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    results = []
    for doc_id, score in ranked[:top_k]:
        snippet = get_snippet(documents[doc_id], query)
        results.append({"doc_id": doc_id, "score": round(score, 4), "snippet": snippet})
    return results


In [14]:
# Exemple
if __name__ == "__main__":
    query_example = "Amoxicilline"
    print(f"\n🔎 Recherche : {query_example}")
    results = search(query_example, top_k=5)
    for r in results:
        print(f"\n📄 {r['doc_id']} | score={r['score']}")
        print(f"Extrait : {r['snippet']}\n")


🔎 Recherche : Amoxicilline

📄 1.md | score=0.4153
Extrait : .../extrait#tab-notice) ANSM - Mis à jour le : 20/11/2023 [1. DENOMINATION DU MEDICAMENT]()[](#top) AMOXICILLINE ACIDE CLAVULANIQUE ALMUS 100 mg/12,5 mg par mL ENFANTS, poudre pour suspension buvable ...


📄 2.md | score=0.204
Extrait : ...EDICAMENT]()[](#top) [AUGMENTIN 1 g/125 mg, poudre pour suspension buvable en sachet-dose (rapport amoxicilline/acide clavulanique : 8/1)]() [2. COMPOSITION QUALITATIVE ET QUANTITATIVE]()[](#top) [...


📄 29.md | score=0.1323
Extrait : ... | | < 30 kg | Association avec deux antibiotiques : INEXIUM 10 mg, amoxicilline 25 mg/kg de poids corporel et clarithromycine 7,5 mg/kg de poids corporel sont tous adm...


📄 28.md | score=0.0848
Extrait : ... et nationaux, ainsi que des recommandations en vigueur. · MOPRAL 20 mg + clarithromycine 500 mg + amoxicilline 1 000 mg, chacun deux fois par jour pendant une semaine, ou · MOPRAL 20 mg + clarithro...


📄 3.md | score=0.0323
Extrait : ...ur le : 11/

In [77]:
qrels = {

    "paracetamol effervescent 1000mg adulte": {"34.md", "37.md", "36.md", "33.md"},

    "tramadol paracetamol association douleur": {"38.md", "36.md", "37.md"},

    "analgesique postop adulte douleur intense": {"34.md", "39.md", "38.md"},

    "amoxicilline suspension enfant": {"1.md", "32.md", "2.md", "29.md"},

    "otite angine sinusite antibiotique": {"4.md", "6.md", "5.md", "2.md"},

    "prednisone betamethasone corticoide": {"17.md", "15.md", "16.md"},

    "antihistaminique allergie saisonniere comprime": {"40.md", "41.md", "39.md", "42.md"},

    "allergie rhinite antihistaminique moderne": {"40.md", "41.md", "39.md"},

    "solution buvable nausees vomissements": {"41.md", "43.md", "7.md", "44.md"},

    "antiemetique vomissements enfant solution": {"41.md", "43.md", "7.md", "44.md"},

    "antipyretique paracetamol adulte": {"37.md", "34.md", "33.md"},

    "traitement douleur chronique tramadol": {"38.md", "36.md", "48.md", "34.md"},

    "antihistaminique rhinite adulte": {"40.md", "39.md", "41.md"},

    "traitement vomissements solution buvable": {"41.md", "43.md", "7.md"},

    "antispasmodique traitement douleurs abdominales": {"24.md", "31.md", "25.md", "30.md"},

    "paracetamol adulte douleur fievre": {"37.md", "34.md", "33.md"},

    "ains adulte douleur fievre": {"18.md", "10.md", "13.md", "11.md"},

    "antibiotique enfant otite angine": {"4.md", "6.md", "5.md", "2.md"},

    "antihistaminique cetirizine loratadine": {"40.md", "41.md", "39.md"}
}


In [78]:
all_results = {}

for query in qrels.keys():
    results = search(query, top_k=5)

    # Si search() retourne None ou un type inattendu
    if results is None or not isinstance(results, list):
        print(f"⚠️ Aucun résultat pour la requête : {query}")
        all_results[query] = []
        continue

    # Extraire doc_id en vérifiant qu’il existe
    retrieved_docs = []
    for r in results:
        if "doc_id" in r:
            retrieved_docs.append(r["doc_id"])
        else:
            print(f"⚠️ Résultat sans doc_id ignoré pour la requête : {query}")

    all_results[query] = retrieved_docs


# Affichage final
for q, docs in all_results.items():
    print(f"\n🔎 Requête : {q}")
    print(f"📄 Classement retourné : {docs if docs else '⚠️ Aucun document trouvé'}")



🔎 Requête : paracetamol effervescent 1000mg adulte
📄 Classement retourné : ['34.md', '37.md', '36.md', '33.md', '19.md']

🔎 Requête : tramadol paracetamol association douleur
📄 Classement retourné : ['38.md', '36.md', '37.md', '10.md', '11.md']

🔎 Requête : analgesique postop adulte douleur intense
📄 Classement retourné : ['34.md', '39.md', '17.md', '38.md', '36.md']

🔎 Requête : amoxicilline suspension enfant
📄 Classement retourné : ['1.md', '32.md', '2.md', '29.md', '19.md']

🔎 Requête : otite angine sinusite antibiotique
📄 Classement retourné : ['4.md', '6.md', '29.md', '5.md', '2.md']

🔎 Requête : prednisone betamethasone corticoide
📄 Classement retourné : ['17.md', '15.md', '16.md']

🔎 Requête : antihistaminique allergie saisonniere comprime
📄 Classement retourné : ['40.md', '41.md', '39.md', '42.md', '35.md']

🔎 Requête : allergie rhinite antihistaminique moderne
📄 Classement retourné : ['40.md', '41.md', '39.md', '35.md', '42.md']

🔎 Requête : solution buvable nausees vomisseme

In [79]:
print(qrels)


def precision_at_k(retrieved, relevant, k):
    retrieved_k = retrieved[:k]
    return len([d for d in retrieved_k if d in relevant]) / k

def recall_at_k(retrieved, relevant, k):
    if len(relevant) == 0:
        return 0
    retrieved_k = retrieved[:k]
    return len([d for d in retrieved_k if d in relevant]) / len(relevant)

def f1_score(p, r):
    if p + r == 0:
        return 0
    return 2 * p * r / (p + r)

def average_precision(retrieved, relevant):
    ap = 0
    hit = 0
    for i, doc in enumerate(retrieved, start=1):
        if doc in relevant:
            hit += 1
            ap += hit / i
    return ap / len(relevant)

def mean_average_precision(all_results, qrels):
    aps = [average_precision(all_results[q], qrels[q]) for q in qrels]
    return sum(aps) / len(aps)


{'paracetamol effervescent 1000mg adulte': {'37.md', '34.md', '36.md', '33.md'}, 'tramadol paracetamol association douleur': {'37.md', '36.md', '38.md'}, 'analgesique postop adulte douleur intense': {'39.md', '34.md', '38.md'}, 'amoxicilline suspension enfant': {'1.md', '2.md', '32.md', '29.md'}, 'otite angine sinusite antibiotique': {'2.md', '4.md', '6.md', '5.md'}, 'prednisone betamethasone corticoide': {'17.md', '16.md', '15.md'}, 'antihistaminique allergie saisonniere comprime': {'42.md', '39.md', '41.md', '40.md'}, 'allergie rhinite antihistaminique moderne': {'39.md', '41.md', '40.md'}, 'solution buvable nausees vomissements': {'43.md', '44.md', '41.md', '7.md'}, 'antiemetique vomissements enfant solution': {'43.md', '44.md', '41.md', '7.md'}, 'antipyretique paracetamol adulte': {'37.md', '34.md', '33.md'}, 'traitement douleur chronique tramadol': {'38.md', '48.md', '36.md', '34.md'}, 'antihistaminique rhinite adulte': {'39.md', '41.md', '40.md'}, 'traitement vomissements solutio

In [80]:


print("=== Évaluation du SRI (modèle vectoriel TF-IDF) ===\n")

for query in qrels.keys():
    retrieved = all_results[query]
    relevant = qrels[query]
    k=5
    P = precision_at_k(retrieved, relevant, k)
    R = recall_at_k(retrieved, relevant, k)
    F1 = f1_score(P, R)
    AP = average_precision(retrieved, relevant)

    print(f"🔎 Requête : {query}")
    print(f"  Precision@{k} = {P:.3f}")
    print(f"  Recall@{k}    = {R:.3f}")
    print(f"  F1-score      = {F1:.3f}")
    print(f"  Average Precision (AP) = {AP:.3f}\n")

MAP = mean_average_precision(all_results, qrels)
print(f"📊 MAP globale = {MAP:.3f}")


=== Évaluation du SRI (modèle vectoriel TF-IDF) ===

🔎 Requête : paracetamol effervescent 1000mg adulte
  Precision@5 = 0.800
  Recall@5    = 1.000
  F1-score      = 0.889
  Average Precision (AP) = 1.000

🔎 Requête : tramadol paracetamol association douleur
  Precision@5 = 0.600
  Recall@5    = 1.000
  F1-score      = 0.750
  Average Precision (AP) = 1.000

🔎 Requête : analgesique postop adulte douleur intense
  Precision@5 = 0.600
  Recall@5    = 1.000
  F1-score      = 0.750
  Average Precision (AP) = 0.917

🔎 Requête : amoxicilline suspension enfant
  Precision@5 = 0.800
  Recall@5    = 1.000
  F1-score      = 0.889
  Average Precision (AP) = 1.000

🔎 Requête : otite angine sinusite antibiotique
  Precision@5 = 0.800
  Recall@5    = 1.000
  F1-score      = 0.889
  Average Precision (AP) = 0.887

🔎 Requête : prednisone betamethasone corticoide
  Precision@5 = 0.600
  Recall@5    = 1.000
  F1-score      = 0.750
  Average Precision (AP) = 1.000

🔎 Requête : antihistaminique allergie s

In [81]:
# ====== Résumé global du système ======

import numpy as np

all_precisions = []
all_recalls = []
all_f1 = []
all_AP = []

for query in qrels.keys():
    retrieved = all_results[query]
    relevant = qrels[query]

    P = precision_at_k(retrieved, relevant, k)
    R = recall_at_k(retrieved, relevant, k)
    F1 = f1_score(P, R)
    AP = average_precision(retrieved, relevant)

    all_precisions.append(P)
    all_recalls.append(R)
    all_f1.append(F1)
    all_AP.append(AP)

print("\n=== Résumé global de l'évaluation ===")
print(f"Moyenne Precision@{k} = {np.mean(all_precisions):.3f}")
print(f"Moyenne Recall@{k}    = {np.mean(all_recalls):.3f}")
print(f"Moyenne F1-score      = {np.mean(all_f1):.3f}")
print(f"Moyenne AP            = {np.mean(all_AP):.3f}")
print(f"MAP globale           = {MAP:.3f}")



=== Résumé global de l'évaluation ===
Moyenne Precision@5 = 0.695
Moyenne Recall@5    = 0.987
Moyenne F1-score      = 0.811
Moyenne AP            = 0.973
MAP globale           = 0.973
